# Feature engineering 

In [1]:
import pandas as pd

In [2]:

ais_train = pd.read_csv('ais_train.csv', sep='|')
ports = pd.read_csv('ports.csv', sep='|')
vessels = pd.read_csv('vessels.csv', sep='|')

In [3]:
# Create a new dataframe called features
features = pd.DataFrame()

# Add selected columns from ais_train to the features dataframe
features['time'] = ais_train['time']
features['cog'] = ais_train['cog']
features['sog'] = ais_train['sog']
features['rot'] = ais_train['rot']
features['navstat'] = ais_train['navstat']
features['longitude'] = ais_train['longitude']
features['latitude'] = ais_train['latitude']

# Merge ais_train with ports to get port longitude and latitude
ais_train = ais_train.merge(ports[['portId', 'longitude', 'latitude']], how='left', left_on='portId', right_on='portId', suffixes=('', '_port'))

# Add port longitude and latitude to the features dataframe
features['port_longitude'] = ais_train['longitude_port']
features['port_latitude'] = ais_train['latitude_port']

# Extract temporal features from the TIME variable
ais_train['time'] = pd.to_datetime(ais_train['time'])
features['hour'] = ais_train['time'].dt.hour
features['day_of_week'] = ais_train['time'].dt.dayofweek
features['month'] = ais_train['time'].dt.month
features['year'] = ais_train['time'].dt.year

